In [1]:
import numpy as np
import plotly
import pandas


In [2]:
#shuffle two arrays in unison
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [3]:
#Input E

#error functions
def l1norm(E,l):
    return sum([x**2 for x in E])

#mean absolute error
def mean_absolute_error(E,l):
    return 1/len(E) * sum([np.abs(x) for x in E])

#huber loss
def huber_loss(E, l):
    result = []
    for e in E:
        if e <= l:
            result.append( 1/2 * e**2 )
        elif e > l:
            result.append( l * np.abs(e) - 1/2 * l**2)
    return sum(result)

In [4]:
x = np.random.uniform(-2,2,50)

import plotly.plotly as py
import plotly.graph_objs as go

# Create random data with numpy
import numpy as np
def huberfun(x,h):

    if np.abs(x) <= h:
        return x**2
    if np.abs(x) > h:
        return h*np.abs(x)

x = np.sort(x)
# Create traces
trace0 = go.Scatter(
    x = x,
    y =[ e**2 for e in x],
    mode = 'lines',
    name = 'square'
)
trace1 = go.Scatter(
    x = x,
    y = [ np.abs(y) for y in x],
    mode = 'lines',
    name = 'mean'
)

trace2 = go.Scatter(
    x = x,
    y = [huberfun(y,.1) for y in x] ,
    mode = 'lines',
    name = 'huber .1'
)

trace3 = go.Scatter(
    x = x,
    y = [huberfun(y,4.1) for y in x] ,
    mode = 'lines',
    name = 'huber 4.1'
)
data = [trace0, trace1, trace2, trace3]

py.iplot(data, filename='line-mode')



In [5]:
#line search
n_max = 1.0
r = .6
tolerance = .0001
max_backtrack_iterations = 50
def line_search(w_t, c, g,h):
        n = n_max
        w = w_t
        #this also depends on the loss function why we pass the c
        obj = c(np.matmul(data,w)-np.array(target),h)
        backtrack_iterations = 0
        while backtrack_iterations < max_backtrack_iterations:
            w = np.array(w_t) - n * np.array(g)
            
            if c(np.matmul(data,w)-np.array(target),h) < (obj - tolerance):
                break
            n = r*n
            
            backtrack_iterations += 1
           
        if backtrack_iterations == max_backtrack_iterations - 1:
            print('here')
            return w_t, 0
            
        
        return w, n  

In [6]:
#derivative of the mean absolute error
def mean_absolute_err_g(data,w,target):
    g = []
    errors = (np.matmul(data,w)-np.array(target))
    for p in range(0, len(data[0])):
        sums = 0
        for idx, e in enumerate(errors):
            if e > 0:
                sums += data[idx][p]   
            if e < 0:
                sums -= data[idx][p]
            if e == 0:
                sums += 0
                    
        g.append(sums)
    return g

#derivative of huber loss
def huber_loss_g(data,w,target,h):
    g = []
    errors = (np.matmul(data,w)-np.array(target))
    for p in range(0, len(data[0])):
        sums = 0
        for idx, e in enumerate(errors):
            
            if np.abs(e) <= h:
                
                sums += e * data[idx][p]
            
            if np.abs(e) > h:
            
                if e > 0:
                    sums += h*data[idx][p]   
                if e < 0:
                    sums -= h*data[idx][p]
                if e == 0:
                    sums += 0
                    
        g.append(sums)
    return g






In [7]:
list(np.random.randn(1,3)[0])

[-0.9110630235342279, 0.2708261141812398, 0.13570731441856598]

In [8]:
#batch  gradient descent implementation
def batch_gradient(data,target,errfun,h):
    #initialize w
    w = list(np.random.randn(1,len(data[0]))[0])
    #super large int (inf)
    err = 5000000000000
    tolerance = .0001
    dim = len(data[0])
    max_iterations = 1000
    iteration = 0
    
    while err > tolerance and iteration < max_iterations:
        err = errfun(np.matmul(data,w)-np.array(target),h)
        #gradient depends on the loss function used
        if errfun == l1norm:
            g =  np.matmul(np.transpose(data) ,(np.matmul(data,w)-np.array(target)))
        if errfun == mean_absolute_error:
            g = mean_absolute_err_g(data,w,target)
        #huber loss is a combination of the l1norm and absolute so the gradient is piecewise for | y_p - y_t | <= h   
        if errfun == huber_loss:
            g = huber_loss_g(data,w,target,h)      
        #insert line search here for the step size  
        #w_ls,n = line_search(w, errfun, g,h)
        n = .001
        #print(n)
        w = np.array(w) - n * np.array(g)
        iteration += 1
    return w

In [9]:
#Stochastic gradient descent
def stochastic_gradient(data,target,errfun,epochs):

    #initialize w
    
    w = list(np.random.randn(1,len(data[0]))[0])
    

    #stochastic gradient descent implementation 
    hub_coef = .9
    for i in range(1,epochs):
        data, target = unison_shuffled_copies(np.array(data),np.array(target))
        #number of epochs in 
        for j in range(0, len(data)):
            obj = np.matmul(np.transpose(data[j]), w) - target[j]
            if errfun == l1norm:
                g = obj * data[j]
            if errfun == mean_absolute_error:
                if obj > 0:
                    g = data[j]
                if obj < 0:
                    g = -data[j]
                if obj == 0:
                    g = 0
            if errfun == huber_loss:
                if obj < hub_coef:
                    g = obj * data[j]
                if obj >= hub_coef:
                    if obj > 0:
                        g = hub_coef*data[j]
                    if obj < 0:
                        g = -hub_coef*data[j]
                    if obj == 0:
                        g = 0
            n = 1/i
            w = np.array(w) - n * np.array(g)   
            
    return w

In [10]:
def analytic(data,target):
    
    s1 = np.linalg.inv(np.matmul(np.transpose(data),np.array(data)))
    s2 = np.matmul(s1,np.transpose(data))
    s3 = np.matmul(s2, target)
    coef_pred = s3
    
    return coef_pred

In [11]:
#####testing
#Problem 5a

rez5a1 = []
#analyti square loss
print('5a1')
for i in range(0,1000):
    X = np.random.uniform(-2,2,50)
    e = np.random.normal(0, 4, 50)

    Y = 2 + 3*X + e
    intercept = [1]*50

    data = list(zip(intercept,X))
    target = Y
    w = analytic(data,target)
    #w = batch_gradient(data,target,huber_loss,8000)

    d = []
    rez5a1.append(w[1])
#Problem 5b batch square loss
rez5a2 = []
print('5a2')
for i in range(0,1000):
    X = np.random.uniform(-2,2,50)
    e = np.random.normal(0, 4, 50)
    Y = 2 + 3*X + e
    intercept = [1]*50
    data = list(zip(intercept,X))
    target = Y
    w = batch_gradient(data,target,l1norm,8000)
    rez5a2.append(w[1])
#Problem 5c stochastic_square loss
rez5a3 = []
print('5a3')
for i in range(0,1000):
    X = np.random.uniform(-2,2,50)
    e = np.random.normal(0, 4, 50)

    Y = 2 + 3*X + e
    intercept = [1]*50

    data = list(zip(intercept,X))
    target = Y
    w = stochastic_gradient(data,target,l1norm,100)
    rez5a3.append(w[1])
    
    


5a1
5a2
5a3


In [12]:
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np

df1 = pandas.DataFrame({'i': rez5a1, 'ii': rez5a2, 'iii': rez5a3})

def visualize(df):
    trace1 = go.Histogram(
        x=df['i'],
        opacity=0.75,
        name ='Analytic'
    )
    trace2 = go.Histogram(
        x=df['ii'],
        opacity=0.75,
        name = 'Batch'
    )
    trace3 = go.Histogram(
        x=df['iii'],
        opacity=0.75,
        name = 'Stochastic'
    )
    data = [trace1,trace2,trace3]
    layout = go.Layout(barmode='overlay', title = 'Slope Estimates')
    fig = go.Figure(data=data, layout=layout)
    
    return data,layout

data,layout = visualize(df1)
fig = go.Figure(data=data, layout=layout)
    
py.iplot(fig, filename='overlaid histogram1')

(i) analytical solution, 
(ii) batch
(iii) stochastic gradient descent implemented in Question 4.
Set learning rate α to a small value (say, α = 0.01).

In [13]:
df1.describe()

,i,ii,iii
count,1000.000000,1000.000000,1000.000000
mean,2.981961,2.975315,2.953991
std,0.502551,0.495785,0.503648
min,1.386048,1.500969,1.111369
25%,2.651742,2.648130,2.605460
50%,3.014026,2.970718,2.947256
75%,3.331852,3.309910,3.295971
max,4.502985,4.693927,4.733961


5b) It seems that all three methods tend to be normally distributed about the mean 3. However, the gradient descent method as shown by the statistics has the best approximate of the intercept and also has the smallest standard deviation. the analytical solution did a little better than the batch gradient descent but then I did not experiment with the tolerance or the number of iterations was fairly low.

In [14]:
#####testing
#Problem 5c

rez5c1 = []
#analyti square loss
print('5c1')
for i in range(0,1000):
    X = np.random.uniform(-2,2,50)
    e = np.random.normal(0, 4, 50)
    Y = 2 + 3*X + e
    intercept = [1]*50
    data = list(zip(intercept,X))
    target = Y
    w = analytic(data,target)
    #w = batch_gradient(data,target,huber_loss,8000)
    rez5c1.append(w[1])
#Problem 5b batch square loss
print('5c2')
rez5c2 = []
for i in range(0,1000):
    X = np.random.uniform(-2,2,50)
    e = np.random.normal(0, 4, 50)
    Y = 2 + 3*X + e
    intercept = [1]*50
    data = list(zip(intercept,X))
    target = Y
    w = batch_gradient(data,target,mean_absolute_error,8000)
    d = []
    rez5c2.append(w[1])
#Problem 5c stochastic_square loss
print('5c3')
rez5c3 = []
for i in range(0,1000):
    X = np.random.uniform(-2,2,50)
    e = np.random.normal(0, 4, 50)
    Y = 2 + 3*X + e
    intercept = [1]*50
    data = list(zip(intercept,X))
    target = Y
    w = stochastic_gradient(data,target,huber_loss,100)
    d = []
    rez5c3.append(w[1])
    

5c1
5c2
5c3


In [15]:
df2 = pandas.DataFrame({'i': rez5c1, 'ii': rez5c2, 'iii': rez5c3})

data,layout = visualize(df2)
fig = go.Figure(data=data, layout=layout)
    
py.iplot(fig, filename='overlaid histogram2')

(i) squared loss with the analytical solution, 
(ii) mean absolute error with batch
(iii) Huber loss with batch gradient descent implemented in


In [16]:
df2.describe()

,i,ii,iii
count,1000.000000,1000.000000,1000.000000
mean,3.016011,2.948911,2.998724
std,0.491682,0.602151,0.649229
min,1.629808,0.614442,0.209743
25%,2.668698,2.548641,2.574863
50%,3.003417,2.927568,2.977030
75%,3.328113,3.349467,3.461048
max,4.747849,5.018678,5.319313


Question 4. Set learning rate α to a small value (say, α = 0.01).


#5d) It seems that in this case, the analytical solution outpreformed the rest of the methods. Again, my number of iterations and tolerances were fixed and I did not experiment enough due to run time. 


In [17]:
import random
#Generate 50 random numbers on interval (0,1]
def generate_data():
    k = [np.random.rand() for x in range(0,50)]

    idx = []
    #store index
    for i in range(0,50):
        if k[i] <= .1: idx.append(i)

    X = np.random.uniform(-2,2,50)
    e = np.random.normal(0, 4, 50)

    Y = 2 + 3*X + e
    intercept = [1]*50
    data = list(zip(intercept,X))
    data = [list(x) for x in data]
    for i in idx:
        r = np.random.rand()
        if r > .5:
            data[i][1] = data[i][1] + 1/2*data[i][1]
        if r<= .5:
            data[i][1] = data[i][1] - 1/2*data[i][1]
    return data,Y

In [18]:
#####testing
#Problem 5c
rez5e1 = []
print('5e1')
#analyti square loss
for i in range(0,1000):
    data,target = generate_data()
    w = analytic(data,target)
    rez5e1.append(w[1])
#Problem 5b batch square loss
rez5e2 = []
print('5e2')
for i in range(0,1000):
    data,target = generate_data()
    w = batch_gradient(data,target,mean_absolute_error,8000)
    rez5e2.append(w[1])
#Problem 5c stochastic_square loss
rez5e3 = []
print('5e3')
for i in range(0,1000):
    data,target = generate_data()
    w = stochastic_gradient(data,target,huber_loss,100)
    rez5e3.append(w[1])

5e1
5e2
5e3


In [19]:
df3 = pandas.DataFrame({'i': rez5e1, 'ii': rez5e2, 'iii': rez5e3})

data,layout = visualize(df3)
fig = go.Figure(data=data, layout=layout)
    
py.iplot(fig, filename='overlaid histogram3')

In [20]:
df3.describe()

,i,ii,iii
count,1000.000000,1000.000000,1000.000000
mean,2.921184,2.905721,2.915388
std,0.499540,0.641434,1.378989
min,0.440308,0.610908,-35.428008
25%,2.596092,2.492900,2.510411
50%,2.917960,2.925696,2.953779
75%,3.267428,3.339575,3.406163
max,4.472105,4.813922,5.470456


5f) According to the statistics the closest approximation was achieved by the analytical solution making me believe that my choice of iterations and tolerance needs improvement as well as more experimentation with the huber coeffiecnt. According to 
the book, the huber loss function should be less sensitive to outliers than the other methods.